# Search and download data from Digtal Africa

https://docs.digitalearthafrica.org/en/latest/data_specs/Landsat_WOfS_specs.html#

In [9]:
import sys, os, importlib
import rasterio, boto3

import pandas as pd
import geopandas as gpd

from botocore import UNSIGNED
from botocore.config import Config
from GOSTRocks.misc import tPrint

bucket = 'deafrica-services'
prefix = 'wofs_ls' #'wofs_ls_summary_annual'
region = 'af-south-1'
s3client = boto3.client('s3', region_name='af-south-1', config=Config(signature_version=UNSIGNED))

In [10]:
# Define your input variables
iso3 = 'MWI'
year = 2020
product = '_frequency.tif' #ount_clear.tif, _count_wet.tif
out_folder = f"/home/wb411133/temp/{iso3}"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

In [11]:
#Read in extents of output tiles
in_extents = gpd.read_file("wofs_ls_summary_alltime-regions-deafrica-data.geojson")
in_extents['COL'] = in_extents['region_code'].apply(lambda x: int(x.split("_")[0]) + 181) # These additions transform numbers for downloads
in_extents['ROW'] = in_extents['region_code'].apply(lambda x: int(x.split("_")[1]) + 77)
in_extents['COL_ROW'] = in_extents.apply(lambda x: f"{x['COL']}_{x['ROW']}", axis=1)

#Read in country extent and select intersecting tiles
adm0 = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
sel_extent = adm0.loc[adm0['iso_a3'] == iso3]
sel_tiles = in_extents.loc[in_extents.intersects(sel_extent.unary_union)]

In [20]:
in_extents

,region_code,label,count,geometry,COL,ROW,COL_ROW
0,-10_10,-10_10,1,"POLYGON ((-9.94960 8.30588, -8.95464 8.30588, ...",171,87,171_87
1,-10_11,-10_11,1,"POLYGON ((-9.94960 9.06690, -8.95464 9.06690, ...",171,88,171_88
2,-10_12,-10_12,1,"POLYGON ((-9.94960 9.82949, -8.95464 9.82949, ...",171,89,171_89
3,-10_13,-10_13,1,"POLYGON ((-9.94960 10.59379, -8.95464 10.59379...",171,90,171_90
4,-10_14,-10_14,1,"POLYGON ((-9.94960 11.35996, -8.95464 11.35996...",171,91,171_91
...,...,...,...,...,...,...,...
4389,9_5,9_5,1,"POLYGON ((8.95464 4.51961, 9.94960 4.51961, 9....",190,82,190_82
4390,9_6,9_6,1,"POLYGON ((8.95464 5.27481, 9.94960 5.27481, 9....",190,83,190_83
4391,9_7,9_7,1,"POLYGON ((8.95464 6.03090, 9.94960 6.03090, 9....",190,84,190_84
4392,9_8,9_8,1,"POLYGON ((8.95464 6.78802, 9.94960 6.78802, 9....",190,85,190_85


In [13]:
# Loop through the S3 bucket and get all the keys for files that are .tif 
more_results = True
try:
    del(token)
except:
    pass
loops = 0
good_res = []
while more_results:
    print(f"Completed loop: {loops}")
    if loops > 0:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix, ContinuationToken=token)
    else:
        objects = s3client.list_objects_v2(Bucket=bucket, Prefix=prefix)
    more_results = objects['IsTruncated']
    if more_results:
        token = objects['NextContinuationToken']
    loops += 1
    for res in objects['Contents']:
        if res['Key'].endswith('.tif'):
            good_res.append(res)

Completed loop: 0
Completed loop: 1
Completed loop: 2
Completed loop: 3
Completed loop: 4
Completed loop: 5
Completed loop: 6
Completed loop: 7
Completed loop: 8
Completed loop: 9
Completed loop: 10
Completed loop: 11
Completed loop: 12
Completed loop: 13
Completed loop: 14
Completed loop: 15
Completed loop: 16
Completed loop: 17
Completed loop: 18
Completed loop: 19
Completed loop: 20
Completed loop: 21
Completed loop: 22
Completed loop: 23
Completed loop: 24
Completed loop: 25
Completed loop: 26
Completed loop: 27
Completed loop: 28
Completed loop: 29
Completed loop: 30
Completed loop: 31
Completed loop: 32
Completed loop: 33
Completed loop: 34
Completed loop: 35
Completed loop: 36
Completed loop: 37
Completed loop: 38
Completed loop: 39
Completed loop: 40
Completed loop: 41
Completed loop: 42
Completed loop: 43
Completed loop: 44
Completed loop: 45
Completed loop: 46
Completed loop: 47
Completed loop: 48
Completed loop: 49
Completed loop: 50
Completed loop: 51
Completed loop: 52
Com

Completed loop: 416
Completed loop: 417
Completed loop: 418
Completed loop: 419
Completed loop: 420
Completed loop: 421
Completed loop: 422
Completed loop: 423
Completed loop: 424
Completed loop: 425
Completed loop: 426
Completed loop: 427
Completed loop: 428
Completed loop: 429
Completed loop: 430
Completed loop: 431
Completed loop: 432
Completed loop: 433
Completed loop: 434
Completed loop: 435
Completed loop: 436
Completed loop: 437
Completed loop: 438
Completed loop: 439
Completed loop: 440
Completed loop: 441
Completed loop: 442
Completed loop: 443
Completed loop: 444
Completed loop: 445
Completed loop: 446
Completed loop: 447
Completed loop: 448
Completed loop: 449
Completed loop: 450
Completed loop: 451
Completed loop: 452
Completed loop: 453
Completed loop: 454
Completed loop: 455
Completed loop: 456
Completed loop: 457
Completed loop: 458
Completed loop: 459
Completed loop: 460
Completed loop: 461
Completed loop: 462
Completed loop: 463
Completed loop: 464
Completed loop: 465


Completed loop: 826
Completed loop: 827
Completed loop: 828
Completed loop: 829
Completed loop: 830
Completed loop: 831
Completed loop: 832
Completed loop: 833
Completed loop: 834
Completed loop: 835
Completed loop: 836
Completed loop: 837
Completed loop: 838
Completed loop: 839
Completed loop: 840
Completed loop: 841
Completed loop: 842
Completed loop: 843
Completed loop: 844
Completed loop: 845
Completed loop: 846
Completed loop: 847
Completed loop: 848
Completed loop: 849
Completed loop: 850
Completed loop: 851
Completed loop: 852
Completed loop: 853
Completed loop: 854
Completed loop: 855
Completed loop: 856
Completed loop: 857
Completed loop: 858
Completed loop: 859
Completed loop: 860
Completed loop: 861
Completed loop: 862
Completed loop: 863
Completed loop: 864
Completed loop: 865
Completed loop: 866
Completed loop: 867
Completed loop: 868
Completed loop: 869
Completed loop: 870
Completed loop: 871
Completed loop: 872
Completed loop: 873
Completed loop: 874
Completed loop: 875


Completed loop: 1225
Completed loop: 1226
Completed loop: 1227
Completed loop: 1228
Completed loop: 1229
Completed loop: 1230
Completed loop: 1231
Completed loop: 1232
Completed loop: 1233
Completed loop: 1234
Completed loop: 1235
Completed loop: 1236
Completed loop: 1237
Completed loop: 1238
Completed loop: 1239
Completed loop: 1240
Completed loop: 1241
Completed loop: 1242
Completed loop: 1243
Completed loop: 1244
Completed loop: 1245
Completed loop: 1246
Completed loop: 1247
Completed loop: 1248
Completed loop: 1249
Completed loop: 1250
Completed loop: 1251
Completed loop: 1252
Completed loop: 1253
Completed loop: 1254
Completed loop: 1255
Completed loop: 1256
Completed loop: 1257
Completed loop: 1258
Completed loop: 1259
Completed loop: 1260
Completed loop: 1261
Completed loop: 1262
Completed loop: 1263
Completed loop: 1264
Completed loop: 1265
Completed loop: 1266
Completed loop: 1267
Completed loop: 1268
Completed loop: 1269
Completed loop: 1270
Completed loop: 1271
Completed loo

Completed loop: 1616
Completed loop: 1617
Completed loop: 1618
Completed loop: 1619
Completed loop: 1620
Completed loop: 1621
Completed loop: 1622
Completed loop: 1623
Completed loop: 1624
Completed loop: 1625
Completed loop: 1626
Completed loop: 1627
Completed loop: 1628
Completed loop: 1629
Completed loop: 1630
Completed loop: 1631
Completed loop: 1632
Completed loop: 1633
Completed loop: 1634
Completed loop: 1635
Completed loop: 1636
Completed loop: 1637
Completed loop: 1638
Completed loop: 1639
Completed loop: 1640
Completed loop: 1641
Completed loop: 1642
Completed loop: 1643
Completed loop: 1644
Completed loop: 1645
Completed loop: 1646
Completed loop: 1647
Completed loop: 1648
Completed loop: 1649
Completed loop: 1650
Completed loop: 1651
Completed loop: 1652
Completed loop: 1653
Completed loop: 1654
Completed loop: 1655
Completed loop: 1656
Completed loop: 1657
Completed loop: 1658
Completed loop: 1659
Completed loop: 1660
Completed loop: 1661
Completed loop: 1662
Completed loo

...

Completed loop: 5526
Completed loop: 5527
Completed loop: 5528
Completed loop: 5529
Completed loop: 5530
Completed loop: 5531
Completed loop: 5532
Completed loop: 5533
Completed loop: 5534
Completed loop: 5535
Completed loop: 5536
Completed loop: 5537
Completed loop: 5538
Completed loop: 5539
Completed loop: 5540
Completed loop: 5541
Completed loop: 5542
Completed loop: 5543
Completed loop: 5544
Completed loop: 5545
Completed loop: 5546
Completed loop: 5547
Completed loop: 5548
Completed loop: 5549
Completed loop: 5550
Completed loop: 5551
Completed loop: 5552
Completed loop: 5553
Completed loop: 5554
Completed loop: 5555
Completed loop: 5556
Completed loop: 5557
Completed loop: 5558
Completed loop: 5559
Completed loop: 5560
Completed loop: 5561
Completed loop: 5562
Completed loop: 5563
Completed loop: 5564
Completed loop: 5565
Completed loop: 5566
Completed loop: 5567
Completed loop: 5568
Completed loop: 5569
Completed loop: 5570
Completed loop: 5571
Completed loop: 5572
Completed loo

Completed loop: 5917
Completed loop: 5918
Completed loop: 5919
Completed loop: 5920
Completed loop: 5921
Completed loop: 5922
Completed loop: 5923
Completed loop: 5924
Completed loop: 5925
Completed loop: 5926
Completed loop: 5927
Completed loop: 5928
Completed loop: 5929
Completed loop: 5930
Completed loop: 5931
Completed loop: 5932
Completed loop: 5933
Completed loop: 5934
Completed loop: 5935
Completed loop: 5936
Completed loop: 5937
Completed loop: 5938
Completed loop: 5939
Completed loop: 5940
Completed loop: 5941
Completed loop: 5942
Completed loop: 5943
Completed loop: 5944
Completed loop: 5945
Completed loop: 5946
Completed loop: 5947
Completed loop: 5948
Completed loop: 5949
Completed loop: 5950
Completed loop: 5951
Completed loop: 5952
Completed loop: 5953
Completed loop: 5954
Completed loop: 5955
Completed loop: 5956
Completed loop: 5957
Completed loop: 5958
Completed loop: 5959
Completed loop: 5960
Completed loop: 5961
Completed loop: 5962
Completed loop: 5963
Completed loo

Completed loop: 6308
Completed loop: 6309
Completed loop: 6310
Completed loop: 6311
Completed loop: 6312
Completed loop: 6313
Completed loop: 6314
Completed loop: 6315
Completed loop: 6316
Completed loop: 6317
Completed loop: 6318
Completed loop: 6319
Completed loop: 6320
Completed loop: 6321
Completed loop: 6322
Completed loop: 6323
Completed loop: 6324
Completed loop: 6325
Completed loop: 6326
Completed loop: 6327
Completed loop: 6328
Completed loop: 6329
Completed loop: 6330
Completed loop: 6331
Completed loop: 6332
Completed loop: 6333
Completed loop: 6334
Completed loop: 6335
Completed loop: 6336
Completed loop: 6337
Completed loop: 6338
Completed loop: 6339
Completed loop: 6340
Completed loop: 6341
Completed loop: 6342
Completed loop: 6343
Completed loop: 6344
Completed loop: 6345
Completed loop: 6346
Completed loop: 6347
Completed loop: 6348
Completed loop: 6349
Completed loop: 6350
Completed loop: 6351
Completed loop: 6352
Completed loop: 6353
Completed loop: 6354
Completed loo

Completed loop: 6699
Completed loop: 6700
Completed loop: 6701
Completed loop: 6702
Completed loop: 6703
Completed loop: 6704
Completed loop: 6705
Completed loop: 6706
Completed loop: 6707
Completed loop: 6708
Completed loop: 6709
Completed loop: 6710
Completed loop: 6711
Completed loop: 6712
Completed loop: 6713
Completed loop: 6714
Completed loop: 6715
Completed loop: 6716
Completed loop: 6717
Completed loop: 6718
Completed loop: 6719
Completed loop: 6720
Completed loop: 6721
Completed loop: 6722
Completed loop: 6723
Completed loop: 6724
Completed loop: 6725
Completed loop: 6726
Completed loop: 6727
Completed loop: 6728
Completed loop: 6729
Completed loop: 6730
Completed loop: 6731
Completed loop: 6732
Completed loop: 6733
Completed loop: 6734
Completed loop: 6735
Completed loop: 6736
Completed loop: 6737
Completed loop: 6738
Completed loop: 6739
Completed loop: 6740
Completed loop: 6741
Completed loop: 6742
Completed loop: 6743
Completed loop: 6744
Completed loop: 6745
Completed loo

Completed loop: 7090
Completed loop: 7091
Completed loop: 7092
Completed loop: 7093
Completed loop: 7094
Completed loop: 7095
Completed loop: 7096
Completed loop: 7097
Completed loop: 7098
Completed loop: 7099
Completed loop: 7100
Completed loop: 7101
Completed loop: 7102
Completed loop: 7103
Completed loop: 7104
Completed loop: 7105
Completed loop: 7106
Completed loop: 7107
Completed loop: 7108
Completed loop: 7109
Completed loop: 7110
Completed loop: 7111
Completed loop: 7112
Completed loop: 7113
Completed loop: 7114
Completed loop: 7115
Completed loop: 7116
Completed loop: 7117
Completed loop: 7118
Completed loop: 7119
Completed loop: 7120
Completed loop: 7121
Completed loop: 7122
Completed loop: 7123
Completed loop: 7124
Completed loop: 7125
Completed loop: 7126
Completed loop: 7127
Completed loop: 7128
Completed loop: 7129
Completed loop: 7130
Completed loop: 7131
Completed loop: 7132
Completed loop: 7133
Completed loop: 7134
Completed loop: 7135
Completed loop: 7136
Completed loo

In [24]:
def get_col(x):
    try:
        return(x.split("/")[2][0:])
    except:
        return(-1)

def get_row(x):
    try:
        return(x.split("/")[3][0:])
    except:
        return(-1)

In [25]:
# Convert the results to a data frame
s3res = pd.DataFrame(good_res)

s3res['COL']  = s3res['Key'].apply(get_col)
s3res['ROW']  = s3res['Key'].apply(get_row)
s3res['COL_ROW'] = s3res.apply(lambda x: f"{x['COL']}_{x['ROW']}", axis=1)
s3res['TYPE'] = s3res['Key'].apply(lambda x: x.split("/")[-1][-14:].split("_")[-1].replace(".tif",""))
s3res['YEAR'] = s3res['Key'].apply(lambda x: x.split("/")[4][:4])
s3res['Size'] = s3res['Size'].apply(lambda x: x/1024/1024)

s3res.head()

,Key,LastModified,ETag,Size,StorageClass,COL,ROW,COL_ROW,TYPE,YEAR
0,wofs_ls/1-0-0/148/072/2005/11/21/wofs_ls_14807...,2021-09-01 04:58:11+00:00,"""865518daa97eb5abada45389ca8275b0""",3.961378,STANDARD,148,072,148_072,water,2005
1,wofs_ls/1-0-0/148/072/2005/12/23/wofs_ls_14807...,2021-09-01 04:06:08+00:00,"""ada110dbc2125e9f887503c1d1eb6099""",3.956423,STANDARD,148,072,148_072,water,2005
2,wofs_ls/1-0-0/148/072/2006/01/24/wofs_ls_14807...,2021-09-01 04:09:59+00:00,"""8baba74aab96bd79fdf4e1e598f5eba0""",4.412727,STANDARD,148,072,148_072,water,2006
3,wofs_ls/1-0-0/148/072/2006/02/25/wofs_ls_14807...,2021-09-01 03:58:38+00:00,"""d960813048736658c36542711939e9f7-2""",8.448545,STANDARD,148,072,148_072,water,2006
4,wofs_ls/1-0-0/148/072/2006/04/30/wofs_ls_14807...,2021-09-01 04:10:04+00:00,"""58b52ddc11ff59c1b56d7d20334d0052""",3.539613,STANDARD,148,072,148_072,water,2006


In [27]:
s3res.to_csv("/home/wb411133/projects/WOFS_Walker/WOFS_AWS_SUMMARY.csv")

In [28]:
s3res['YEAR'].value_counts()

2013    13029
2014    13002
2015    12981
2017    12966
2021    12957
2019    12948
2018    12942
2016    12942
2020    12933
2006    12867
2005    12807
2012    12807
2000    12798
2001    12795
2007    12777
2002    12771
2009    12771
2011    12753
2003    12732
2010    12726
2008    12693
1999    12660
2004    12648
1998    11673
1984    11643
1995    11469
1987    11460
1994    11313
1986    11301
1990    10341
1985     9600
1989     9585
1991     8889
1996     8829
1997     8643
1993     7896
1992     7821
1988     6492
Name: YEAR, dtype: int64

In [7]:
s3res.loc[s3res['TYPE'] == "_frequency.tif"].groupby(['TYPE','YEAR']).sum()

Size     COL     ROW
TYPE           YEAR                              
_frequency.tif 1984   5034.794310  772208  330751
               1985   3446.524641  639464  282077
               1986   4468.735968  748074  323759
               1987   4896.933995  757938  329658
               1988   4011.129354  421407  219596
               1989   5083.858824  638119  272228
               1990   5444.456020  687116  292158
               1991   5079.096058  605646  244675
               1992   4035.532763  536992  214671
               1993   5032.544667  543469  214335
               1994   6394.443888  759567  315073
               1995   7180.230254  768937  319825
               1996   5937.811051  600244  242147
               1997   6468.286936  589870  235685
               1998   8189.410395  775992  329235
               1999   8776.168193  846120  352100
               2000  11041.504107  856574  355109
               2001  11518.240195  856083  355234
               2002  11367.222950  854230  354738
               2003   8726.625243  851336  354045
               2004  10112.095716  844770  352196
               2005  10087.569577  857011  355096
               2006  11253.040037  861624  356412
               2007  11054.769150  854630  354723
               2008  10176.978341  848655  353443
               2009  11676.764560  854479  355083
               2010  10469.536771  851002  353762
               2011  10305.945565  853239  354470
               2012  10186.441853  857097  355611
               2013  19743.274564  871907  361354
               2014  23283.692509  870502  360335
               2015  23158.262785  868977  359784
               2016  24004.376737  866076  359003
               2017  23708.886492  867549  359647
               2018  23899.091836  865960  359031
               2019  23555.587044  866378  359150
               2020  23869.449596  865422  358802
               2021  25024.002780  875422  362039

# Download all data for specific year and type

In [19]:
# Download all time summaries
sel_type = "_frequency.tif"
to_download = s3res.loc[(s3res['TYPE'] == sel_type)]
out_folder = f"/home/wb411133/temp/WOFS_Frequency_AllTime"
if not os.path.exists(out_folder):
    os.makedirs(out_folder)

cnt = 0
for idx, row in to_download.iterrows():
    out_file = os.path.join(out_folder, os.path.basename(row['Key']))
    if not os.path.exists(out_file):
        s3client.download_file(bucket, row['Key'], out_file)
    if cnt % 1000 == 0:
        tPrint(f'{year}: downloading {cnt} of {to_download.shape[0]}')
    cnt += 1

16:32:54	2020: downloading 0 of 4394
16:58:55	2020: downloading 1000 of 4394
17:27:46	2020: downloading 2000 of 4394
17:57:43	2020: downloading 3000 of 4394
18:31:03	2020: downloading 4000 of 4394


In [29]:
# Download annual summaries

for year in range(1984, 2001):
    sel_type = "_frequency.tif"
    to_download = s3res.loc[(s3res['TYPE'] == sel_type) & (s3res['YEAR'] == str(year))]

    out_folder = f"/home/wb411133/temp/WOFS_Frequency_{year}"
    if not os.path.exists(out_folder):
        os.makedirs(out_folder)
        
    cnt = 0
    for idx, row in to_download.iterrows():
        out_file = os.path.join(out_folder, os.path.basename(row['Key']))
        if not os.path.exists(out_file):
            s3client.download_file(bucket, row['Key'], out_file)
        if cnt % 1000 == 0:
            tPrint(f'{year}: downloading {cnt} of {to_download.shape[0]}')
        cnt += 1
        
            

09:12:17	1984: downloading 0 of 3881
09:25:31	1984: downloading 1000 of 3881
09:37:22	1984: downloading 2000 of 3881
09:48:57	1984: downloading 3000 of 3881
09:58:42	1985: downloading 0 of 3200
10:10:49	1985: downloading 1000 of 3200
10:23:38	1985: downloading 2000 of 3200
10:35:26	1985: downloading 3000 of 3200
10:37:52	1986: downloading 0 of 3767
10:48:00	1986: downloading 1000 of 3767
11:02:22	1986: downloading 2000 of 3767
11:12:53	1986: downloading 3000 of 3767
11:22:01	1987: downloading 0 of 3820
11:34:00	1987: downloading 1000 of 3820
11:46:20	1987: downloading 2000 of 3820
11:57:51	1987: downloading 3000 of 3820
12:08:59	1988: downloading 0 of 2164
12:21:15	1988: downloading 1000 of 2164
12:36:32	1988: downloading 2000 of 2164
12:38:40	1989: downloading 0 of 3195
12:50:54	1989: downloading 1000 of 3195
13:04:27	1989: downloading 2000 of 3195
13:18:54	1989: downloading 3000 of 3195
13:22:19	1990: downloading 0 of 3447
13:33:12	1990: downloading 1000 of 3447
13:47:15	1990: downlo

In [ ]:
to_download.iloc[0,0]

# Download  data for specific area

In [ ]:
# Limit s3 results to selected area
cur_s3 = s3res.loc[s3res['COL_ROW'].isin(sel_tiles['COL_ROW'])]
cur_s3 = cur_s3.loc[(cur_s3['YEAR'] == str(year)) & (cur_s3['TYPE'] == product)]

In [ ]:
# Download the results
loop = 0
for idx, row in cur_s3.iterrows():
    out_file = os.path.join(out_folder, os.path.basename(row['Key']))
    s3client.download_file(bucket, row['Key'], out_file)
    loop += 1
    print(f'{loop} of {cur_s3.shape}')